In [134]:
#supressing Pandas future warnings when indexing on datetime : https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import requests
import quiverquant
from configparser import ConfigParser
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')

parser = ConfigParser()
_ = parser.read('notebook.cfg')

nasdaq_auth_key = parser.get('my_api', 'nasd_key')
quant_auth_key = parser.get('my_api', 'quant_key')
alpha_auth_key = parser.get('my_api', 'alpha_key')

### Daily WSB Data from 2018-2021 (wsb_all_tickers)

In [2]:
quiver = quiverquant.quiver(quant_auth_key)

In [12]:
#pull all the historical data for wallstreetbets dataset
wsb_all_tickers = quiver.wallstreetbets()

https://api.quiverquant.com/beta/live/wallstreetbets?count_all=true


In [13]:
wsb_all_tickers.to_csv('wsb_all_tickers.csv')

In [51]:
wsb_all_tickers = pd.read_csv('wsb_all_tickers.csv')

In [52]:
wsb_all_tickers.drop(columns=['Unnamed: 0', 'Time'], inplace=True)
wsb_all_tickers.head()

,Ticker,Count,Sentiment,Date
0,SPY,235,0.067476,2021-12-07
1,TSLA,84,0.048281,2021-12-07
2,NVDA,71,0.037637,2021-12-07
3,AAPL,58,0.034978,2021-12-07
4,GME,49,0.072012,2021-12-07


In [53]:
#turn Date column into pandas datetime type
wsb_all_tickers.Date = pd.to_datetime(wsb_all_tickers.Date)

In [54]:
#check that Date column is now datetime type
wsb_all_tickers.dtypes

Ticker               object
Count                 int64
Sentiment           float64
Date         datetime64[ns]
dtype: object

In [55]:
wsb_all_tickers.set_index('Date', inplace=True)

In [56]:
wsb_all_tickers.sort_index(inplace=True)

In [57]:
print(f'Start Date : {wsb_all_tickers.index.min()}')
print(f'End Date : {wsb_all_tickers.index.max()}')

Start Date : 2018-08-01 00:00:00
End Date : 2021-12-07 00:00:00


### Daily WSB Data for 2021 (wsb_2021_all)

In [58]:
wsb_2021_all = wsb_all_tickers['2021']
wsb_2021_all.to_csv('wsb_2021_all.csv')

C:\Users\brian\AppData\Local\Temp/ipykernel_3672/3468444749.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  wsb_2021_all = wsb_all_tickers['2021']


In [10]:
wsb_2021_all = pd.read_csv('./Data/wsb_2021_all.csv')

In [11]:
wsb_2021_all['Date'] = pd.to_datetime(wsb_2021_all['Date'])

In [12]:
wsb_2021_all.set_index('Date', inplace=True)

### 2021 YTD Aggregated WSB Data (wsb_2021_ytd)

In [13]:
wsb_2021_all.head()

,Ticker,Count,Sentiment
Date,,,
2021-01-01,MOON,3,0.920567
2021-01-01,REGN,3,0.259100
2021-01-01,MRNA,3,0.130133
2021-01-01,NEW,3,0.000300
2021-01-01,NVTA,3,0.318367


In [14]:
wsb_2021_ytd = wsb_2021_all.groupby('Ticker').resample('Y')['Count', 'Sentiment'].mean()

#### Initial ETF Portfolio - Highest WSB Mention Count YTD

In [15]:
#sort by highest count to lowest sentiment
top_tickers = wsb_2021_ytd.sort_values(['Count'], ascending=False)

#filter on only tickers that had an average of at least 25 mentions per month on WSB
top_tickers = top_tickers[top_tickers['Count'] > 25]

#### List of Initial ETF Tickers

In [16]:
tick_list = []
for i in top_tickers.index:
    tick_list.append(i[0])
    #print(i[0])
etf_tickers = tick_list
len(etf_tickers)

61

#### Filter 2021 YTD Aggregated WSB Dataset - ETF Tickers Only

In [17]:
#remove FLS, TFC, WRB, AIZ, NTRS, ABMD, TDG as they seem to be faulty data
wsb_2021_ytd.loc[tick_list].head(50)

,,Count,Sentiment
Ticker,Date,,
GME,2021-12-31,979.709677,0.063382
AMC,2021-12-31,570.911504,0.080734
BB,2021-12-31,427.891496,0.088636
SPY,2021-12-31,385.659824,0.029897
PLTR,2021-12-31,273.753666,0.062931
TSLA,2021-12-31,253.724340,0.056084
NOK,2021-12-31,157.252396,0.114438
CLOV,2021-12-31,133.588785,0.078384
AAPL,2021-12-31,111.560117,0.058536


#### Update Initial ETF Ticker List - Top 10

In [22]:
new_etf_tickers = ['GME', 'AMC', 'BB', 'SPY', 'PLTR', 'TSLA', 'NOK', 'CLOV', 'AAPL', 'AMD']

#### Narrow Down Ticker List Based on Correlations

##### Weekly Average Count and Sentiment Score for each Ticker

In [86]:
wsb_2021_weekly = wsb_2021_all.groupby('Ticker').resample('W')['Count', 'Sentiment'].mean()

In [95]:
wsb_2021_weekly = wsb_2021_weekly.loc[new_etf_tickers]
wsb_2021_weekly.head()

Count  Sentiment
Ticker Date                               
GME    2021-01-03    174.666667   0.168927
       2021-01-10    433.714286   0.043846
       2021-01-17   3616.285714   0.091440
       2021-01-24   2053.142857   0.098702
       2021-01-31  11107.428571   0.073145

In [41]:
wsb_top.loc[('AAPL', '2021-10-10')]
print(f"Count : {round(wsb_top.loc[('AAPL', '2019-11-10')][0],2)}")
print(f"Sentiment : {round(wsb_top.loc[('AAPL', '2019-11-10')][1],4)}")

Count : 14.83
Sentiment : 0.0348


In [ ]:
#wsb_top.reset_index()

In [50]:
#Get total mentions by week
#dfWeek = wsb_all_tickers.groupby([pd.Grouper(key='Date', freq='W-MON'), 'Ticker'])['Count'].sum().reset_index().sort_values('Date')